In [ ]:
import os
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from torchvision import transforms as transforms
import torchvision.transforms.functional as tf

from window.utils.tiling import create_tile_list
from window.models.yolo_for_inference import YoloClass
from window.postprocess.match_truths import match_results_to_truths, match_yolo_to_truths
from window.utils.drawing import draw_results_on_image
from window.utils.truths import windows_truth
from window.utils.inference_windows import process_annotation_df_negative_inference, create_windows_from_yolo, windows_to_whole_im

In [ ]:
#output_base_dir = "/home/cmf21/pytorch_save/GFRC/Bin/rgb_baseline2/"
#output_base_dir = "/home/cmf21/pytorch_save/GFRC/Bin/meta_prediction/"
output_base_dir = "/home/cmf21/pytorch_save/GFRC/Bin/meta_prediction_norm/"

valid_whole_image_dir = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"
truth_file = "/data/old_home_dir/ChrissyF/GFRC/yolo_valid_GFRC_bboxes.csv"

output_dir = output_base_dir + "valid_images_out/"
output_csv = output_dir + "results_matched_nms.csv"

# get all image files
image_files = list(Path(valid_whole_image_dir).glob("*.jpg"))
image_files = [img.name for img in image_files]

# Process truth files
truths = pd.read_csv(truth_file)
truths.loc[:, 'filename'] = [strin.replace('/', '_') for strin in truths.file_loc]

# create yolo model
#saveweightspath = output_base_dir + "testing_save_163.pt"
#saveweightspath = output_base_dir + "testing_save_132.pt"
saveweightspath = output_base_dir + "testing_save_127.pt"
channels_in = 4
yolo_model = YoloClass(wtpath=saveweightspath, channels=channels_in)

# create empty list to store results
results_all_ims = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'confmat', 'tru_box', 'filename'])

In [ ]:
if channels_in > 3:
    image_data = pd.read_csv('/home/cmf21/pytorch_save/GFRC/preds_for_cnn.csv')
    metacolumns = ['prediction_norm']

In [ ]:
for fl in image_files:
    truths_im = truths[truths.filename == fl]
    truths_im_pixels = windows_truth(truths_im)

    # Per image
    print(fl)
    whole_im = cv2.imread(valid_whole_image_dir + fl)
    whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)
    if channels_in > 3:
        whole_im = np.divide(whole_im, 255.0)
        matched_row = image_data[image_data.image_name == fl]
        fillval = matched_row[metacolumns]
        fillvals = np.expand_dims(fillval, axis=0)
        new_channel = np.ones((whole_im.shape[0], whole_im.shape[1], len(metacolumns)))
        new_channel = np.multiply(new_channel, fillvals)
        whole_im = np.dstack((whole_im, new_channel))
        whole_im = np.array(whole_im, dtype=np.float32)

    # create tiles
    tilez = create_tile_list(whole_im)

    # get predictions from yolo
    boxes_whole_im = yolo_model.inference_on_image(tilez, 0.3)
    
    windows_whole_im = windows_to_whole_im(boxes_whole_im)
    
    # calculate results
    iou_threshold = 0.15
    nms_threshold = 0.1
    results_per_im = match_yolo_to_truths(windows_whole_im, truths_im_pixels, iou_threshold, nms_threshold)
    results_per_im['filename'] = fl

    results_all_ims = pd.concat((results_all_ims, results_per_im), axis=0, sort=False)

    # draw results on image
    if channels_in > 3:
        whole_im = whole_im[:, :, 0:3]
        whole_im = whole_im * 255
    image_out = draw_results_on_image(whole_im, results_per_im)
    image_out = cv2.cvtColor(image_out, cv2.COLOR_BGR2RGB)
    cv2.imwrite(output_dir + fl, image_out)


In [ ]:
results_all_ims.to_csv(output_csv, index=False)

print(f'TP: {np.sum(results_all_ims.confmat=="TP")}, FP: {np.sum(results_all_ims.confmat=="FP")}, FN: {np.sum(results_all_ims.confmat=="FN")}')
print("number of images:", len(image_files))

In [ ]:
results_all_ims

In [ ]:
unique_ims = np.unique(results_all_ims.filename)
nperim = []
tpsim = []
for im in unique_ims:
    files = results_all_ims[results_all_ims.filename == im]
    nperim.append(files.shape[0])
    tpsim.append(np.sum(files.confmat == 'TP'))

In [ ]:
unique_ims[np.argmax(nperim)]
# 'Z247_Img14318.jpg'
unique_ims[np.argmax(tpsim)]
'Z124_Img13083.jpg'

In [ ]:
bad_im = unique_ims[np.argmax(nperim)]
#whole_im = cv2.imread(output_dir + bad_im)

from PIL import Image

Image.open(output_dir + bad_im)

In [ ]:
good_im = unique_ims[np.argmax(tpsim)]
#whole_im = cv2.imread(output_dir + good_im)

from PIL import Image

Image.open(output_dir + good_im)

In [ ]:
yolo_plus_results_loc = '/home/cmf21/pytorch_save/GFRC/Bin/post_processed/pos_from_yolo/rgb_baseline2_incep/results_post_window_classifier.csv'

import pandas as pd

yolo_plus_results = pd.read_csv(yolo_plus_results_loc)
yolo_plus_results

In [ ]:
import numpy as np

unique_ims = np.unique(yolo_plus_results.filename)
nperim = []
tpsim = []
fnsim = []
for im in unique_ims:
    files = yolo_plus_results[yolo_plus_results.filename == im]
    nperim.append(files.shape[0])
    tpsim.append(np.sum(files.confmat == 'TP'))
    fnsim.append(np.sum(files.confmat == 'FN'))

In [ ]:
print(unique_ims[np.argmax(nperim)])
# Z247_Img14318.jpg
print(unique_ims[np.argmax(tpsim)])
# Z124_Img13083.jpg
print(unique_ims[np.argmax(fnsim)])
# Z138_Img02887.jpg

In [ ]:
from PIL import Image

output_dir = output_base_dir = "/home/cmf21/pytorch_save/GFRC/Bin/post_processed/pos_from_yolo/rgb_baseline2_incep/valid_results_on_image/"
med_im = unique_ims[np.argmax(fnsim)]
whole_im = Image.open(output_dir + med_im)
whole_im